In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout

In [2]:
data = pd.read_csv("Churn_Modelling.csv")

In [3]:
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [10]:
X = data.iloc[:, 3: 13]
y = data.iloc[:, 13]

In [11]:
geography=pd.get_dummies(X['Geography'], drop_first= True)
gender=pd.get_dummies(X['Gender'], drop_first= True)

In [12]:
X=pd.concat([X,geography,gender],axis=1)

In [13]:
X=X.drop(['Geography','Gender'],axis=1)

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [15]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [16]:
classifier=Sequential()
#Adding the Input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'he_uniform',activation='relu',input_dim=11))
#Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'he_uniform',activation='relu'))
#Adding the third hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'he_uniform',activation='relu'))
#Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'glorot_uniform',activation='sigmoid'))
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 163
Trainable params: 163
Non-trainable params: 0
_________________________________________________________________


In [17]:
#Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [21]:
#Fitting the ANN to training set
model_history=classifier.fit(X_train,y_train,validation_split=0.33,batch_size=10, epochs=100)

Epoch 1/100
536/536 [==============================] - 1s 1ms/step - loss: 0.6409 - accuracy: 0.6867 - val_loss: 0.5173 - val_accuracy: 0.7910
Epoch 2/100
536/536 [==============================] - 1s 1ms/step - loss: 0.4985 - accuracy: 0.7962 - val_loss: 0.4953 - val_accuracy: 0.7910
Epoch 3/100
536/536 [==============================] - 1s 1ms/step - loss: 0.4768 - accuracy: 0.7960 - val_loss: 0.4704 - val_accuracy: 0.7910
Epoch 4/100
536/536 [==============================] - 1s 1ms/step - loss: 0.4513 - accuracy: 0.7968 - val_loss: 0.4507 - val_accuracy: 0.7967
Epoch 5/100
536/536 [==============================] - 1s 1ms/step - loss: 0.4358 - accuracy: 0.8031 - val_loss: 0.4412 - val_accuracy: 0.8042
Epoch 6/100
536/536 [==============================] - 1s 1ms/step - loss: 0.4260 - accuracy: 0.8095 - val_loss: 0.4378 - val_accuracy: 0.8073
Epoch 7/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4192 - accuracy: 0.8162 - val_loss: 0.4315 - val_accuracy: 0.8129

In [22]:
#Predicting the test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [23]:
#confusion matrix and the metrics
from sklearn.metrics import confusion_matrix
c=confusion_matrix(y_test,y_pred)

from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_test)

In [24]:
print(c,score)

[[1534   73]
 [ 203  190]] 0.862


## Lets do the hyper parameter tuning to get the best hyper parameters

In [25]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dense,Activation,Embedding,Flatten,LeakyReLU,BatchNormalization
from keras.activations import relu, sigmoid
from keras.layers import Dropout

In [26]:
def create_model(layers,activation):
    model=Sequential()
    for i,nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
    model.add(Dense(units=1,kernel_initializer='uniform',activation='sigmoid'))
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model
model=KerasClassifier(build_fn=create_model, verbose=0)

In [27]:
layers=[(20,),(40,20),(20,10),(30,20),(25,20)]
activations=['sigmoid','relu']
param_grid = dict(layers= layers,activation= activations,batch_size= [128,256],epochs= [30])

grid=GridSearchCV(estimator=model,param_grid=param_grid,cv=5)

In [28]:
grid_result=grid.fit(X_train,y_train)

In [29]:
print(grid_result.best_score_,grid_result.best_params_)

0.8539999842643737 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': (40, 20)}


In [30]:
pred_y=grid.predict(X_test)
y_pred=(pred_y>0.5)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [31]:
from sklearn.metrics import confusion_matrix,accuracy_score
c1=confusion_matrix(y_pred,y_test)
score=accuracy_score(y_pred,y_test)
print(c1,score)

[[1563  240]
 [  44  153]] 0.858
